In [10]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/BN4101-Final-Product-main/

!pip install mediapipe
!pip install jupyter-dash

#------------------------SQL-----------------------------------------
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/

#Emergency fix
try:
  %sql CREATE DATABASE predictionbase
except:
  %sql SHOW DATABASES
  print("Database created")

try:
  %sql USE predictionbase
except:
  %sql SHOW TABLES

#Dropping the table in case it has been previously defined for demonstration purpose 
try:
  %sql DROP TABLE all_predictions
except:
  print("Table drop executed")
## creating the 'all_predictions' table again with the 'PRIMARY KEY'
## prediction level from 0 to 3
try:
  %sql CREATE TABLE all_predictions (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, staff_id VARCHAR(255), \
  student_id VARCHAR(255), module_code VARCHAR(255), class_id VARCHAR(255), frame_num INT(11), pred_lvl INT(11))
except:
  %sql SELECT * FROM all_predictions
#-------------------------------------------------------------------------


import os
from os import listdir
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from google.colab import files

from IPython.display import Image
import pandas as pd
import numpy as np
import cv2
import time

import mediapipe as mp

import NusFypFeatureEngineering as nffe

from keras.models import model_from_json

## For graphing
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque
##

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

def imageProcess(image, subject_name, sequence, label, keypoints):

    # For static images:
    face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        min_detection_confidence=0.5)
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

    # Convert the BGR image to RGB before processing.
    try:
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if keypoints == {}:

            keypoints["Name"] = [subject_name]
            keypoints["Sequence"] = [sequence]
            keypoints["Target"] = [label]

            for i, data_point in enumerate(results.multi_face_landmarks[0].landmark):
                keypoints['X'+str(i)] = [data_point.x]
                keypoints['Y'+str(i)] = [data_point.y]

        else:
            for i, data_point in enumerate(results.multi_face_landmarks[0].landmark):
                keypoints['X'+str(i)] += [data_point.x]
                keypoints['Y'+str(i)] += [data_point.y]

            keypoints["Name"] += [subject_name]
            keypoints["Sequence"] += [sequence]
            keypoints["Target"] += [label]

    except:
        print("No face detected.")

        for i in range(468):
            keypoints['X'+str(i)] += ["No Face"]
            keypoints['Y'+str(i)] += ["No Face"]

        keypoints["Name"] += [subject_name]
        keypoints["Sequence"] += [sequence]
        keypoints["Target"] += [label]

    face_mesh.close()

    return keypoints

def laggingDf(df, maxLag):
    tempDf = df
    for lag in range(1, maxLag+1):
        # Lagging the dataframe and subtract from previous row
        laggedDiffDf = pd.DataFrame(tempDf["Name"]).join(tempDf.groupby(['Name']).shift(
            0).add_prefix('lag'+str(lag)+"_")).groupby(["Name"]).diff(periods=lag)
        # Adding the wrangled rows back into original dataframe
        df = df.join(laggedDiffDf[laggedDiffDf.columns[2:]])
    return df.dropna()

def mergeDf(df1, df2):
    a = np.intersect1d(fe_ele_train_dataset.columns, dfCleaned.columns)

    df1 = df1.drop(a, axis=1)
    df3 = df1.join(df2)

    return df3.dropna()

######################################################
################# GRAPHING IN REAL-TIME ##############
df = 0 #Initialize the df

X = deque()
X.append(1)
Y = deque()
Y.append(1)


app = JupyterDash(__name__)
app.layout = html.Div(
    [
        dcc.Graph(id='live-graph', animate=True),
        dcc.Interval(
            id='graph-update',
            interval=1000,
            n_intervals = 0
        ),
    ]
)

@app.callback(Output('live-graph', 'figure'),
        [Input('graph-update', 'n_intervals')])


def update_graph_scatter(n):
    X.append(df["frame_num"][len(X)])
    Y.append(df["pred_lvl"][len(Y)])

    data = plotly.graph_objs.Scatter(
            x=list(X),
            y=list(Y),
            name='Scatter',
            mode= 'lines+markers'
            )

    return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),
                                                yaxis=dict(range=[min(Y),max(Y)]),)}

app.run_server(mode='external')
######################################################

i = 0

maxLag = 1   #Looking into how many seconds we want to look backwards

#Feature Engineering preparation
x_fea,y_fea = nffe.sepXY([],[])

x_fea_eng,x_fea_alone = nffe.sepEng(x_fea,[],[])
y_fea_eng,y_fea_alone = nffe.sepEng(y_fea,[],[]) 


#Loading deep learning model
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

#Loading final filters
finalFil = pd.read_csv("final_features.csv")

#Class information
StudentID = "A0168781N"
StaffID = "00001"
ModuleID = "BN4301"
ClassID = "S1"
frame = 1
predInterval = 10

while True:
  #Denote the start time
  start_time = time.time()

  try:
    keypoints = {}
    #Take the photo the same number of time we want to look backwards
    for i in range(maxLag+1):
      filename = take_photo()
      
      i += 1

      keypoints = imageProcess(cv2.imread("photo.jpg"),"Yong Jun",i,0,keypoints)

      #Wait for one second
      time.sleep(1)

    fe_ele_train_dataset = nffe.addNewFeature(pd.DataFrame(keypoints),x_fea_eng)
    fe_ele_train_dataset = nffe.addNewFeature(fe_ele_train_dataset,y_fea_eng)

    fe_ele_train_dataset = fe_ele_train_dataset[["Target"]+nffe.fea80()]

    dfCleaned = laggingDf(pd.DataFrame(keypoints),maxLag)

    talliedDf = mergeDf(fe_ele_train_dataset,dfCleaned)

    preds = loaded_model.predict(talliedDf[finalFil["0"]],batch_size = 128)

    #Insert into SQL
    values = (StaffID,StudentID,ModuleID,ClassID,frame,np.argmax(preds))

    try:
      %sql INSERT INTO all_predictions (staff_id, student_id, module_code, class_id, frame_num, pred_lvl) VALUES :values
    except:
      %sql SELECT * FROM all_predictions

    print("Engagement Level is",np.argmax(preds))

    df1 = %sql SELECT frame_num, pred_lvl FROM all_predictions

    if frame == 3:
      prediction = sum(df1[-3:]['pred_lvl'])/3
      df=pd.DataFrame({"frame_num":[frame-2],"pred_lvl":[prediction]})
    elif frame > 3:
      prediction = sum(df1[-3:]['pred_lvl'])/3
      df = df.append(pd.DataFrame({"frame_num":[frame],"pred_lvl":[prediction]})).reset_index(drop=True)

    frame += 1

  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

    #Insert into SQL
    values = (StaffID,StudentID,ModuleID,ClassID,frame,0)

    try:
      %sql INSERT INTO all_predictions (staff_id, student_id, module_code, class_id, frame_num, pred_lvl) VALUES :values
    except:
      %sql SELECT * FROM all_predictions

    print("Engagement Level is",0)

    df1 = %sql SELECT frame_num, pred_lvl FROM all_predictions

    if frame == 3:
      prediction = sum(df1[-3:]['pred_lvl'])/3
      df=pd.DataFrame({"frame_num":[frame-2],"pred_lvl":[prediction]})
    elif frame > 3:
      prediction = sum(df1[-3:]['pred_lvl'])/3
      df = df.append(pd.DataFrame({"frame_num":[frame],"pred_lvl":[prediction]})).reset_index(drop=True)

    frame += 1

  try:
    time.sleep(predInterval-(time.time()-start_time))
    print("Going to the next loop")
  except:
    print("Going to the next loop")






Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/MyDrive/BN4101 Final Product/BN4101-Final-Product/'
/content/gdrive/MyDrive/BN4101 Final Product/BN4101-Final-Product
 * Starting MySQL database server mysqld
   ...done.
ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * mysql+pymysql://root:***@/
(pymysql.err.ProgrammingError) (1007, "Can't create database 'predictionbase'; database exists")
[SQL: CREATE DATABASE predictionbase]
(Background on this error at: http://sqlalche.me/e/13/f405)
 * mysql+pymysql://root:***@/
 * mysql+pymysql://root:***@/
Dash app running on:


<IPython.core.display.Javascript object>

Loaded model from disk


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

No face detected.
'X0'
 * mysql+pymysql://root:***@/
Engagement Level is 0
 * mysql+pymysql://root:***@/
Going to the next loop


<IPython.core.display.Javascript object>

No face detected.
'X0'
 * mysql+pymysql://root:***@/
Engagement Level is 0
 * mysql+pymysql://root:***@/
Going to the next loop


<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored